# VolumeControlProject

In [ ]:





import cv2
import mediapipe as mp
import time
import math
import numpy as np


from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
#volume.GetMute()
#volume.GetMasterVolumeLevel()
volRange=volume.GetVolumeRange()

minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400   #Value may be different
volPer=0



class handDetector():
    def __init__(self, mode=False, maxHands=2, modelComplexity=1,detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.modelComplex = modelComplexity
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,self.modelComplex,
                                        self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,
                                               self.mpHands.HAND_CONNECTIONS)
        return img
    def findPosition(self, img, handNo=0, draw=True):
        lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                # print(id, cx, cy)
                lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
        return lmList
def main():
    pTime = 0
    cTime = 0
    cap = cv2.VideoCapture(0)
    detector = handDetector()
    while True:
        success, img = cap.read()
        img = detector.findHands(img)
        lmlist = detector.findPosition(img)
        #if len(lmList) != 0:
            #print(lmList)
        if len(lmlist) != 0:
            #print(lmlist[4],lmlist[8])
            x1,y1 = lmlist[4][1], lmlist[4][2]
            x2,y2 = lmlist[8][1], lmlist[8][2]
            cx,cy = (x1+x2)//2, (y1+y2)//2
            cv2.circle(img,(x1,y1),15,(255,0,255),cv2.FILLED)
            cv2.circle(img,(x2,y2),15,(255,0,255),cv2.FILLED)
            cv2.line(img,(x1,y1),(x2,y2),(255,0,255),3)
            cv2.circle(img,(cx,cy),15,(255,0,255),cv2.FILLED)
            
            
            length = math.hypot(x2-x1, y2-y1)
            #print(length)
            #Range of Hand is 20-220
            #Volume range -96 - 0
            vol=np.interp(length,[25,220],[minVol,maxVol])
            volBar=np.interp(length,[25,220],[400,150])
            volPer=np.interp(length,[25,220],[0,100])

            

            

            #print(int(length),vol)
            volume.SetMasterVolumeLevel(vol, None)
            if(length<25): #Your minimunm distance between thumb and first finger will be diffrent  
                cv2.circle(img,(x2,y2),15,(0,255,0),cv2.FILLED)

            cv2.rectangle(img,(50,150),(85,400),(0,255,0),3)
            cv2.rectangle(img,(50,int(volBar)),(85,400),(0,255,0),cv2.FILLED)
            cv2.putText(img,f'{int(volPer)}%',(50,450),cv2.FONT_HERSHEY_COMPLEX,
                  1,(0,0,255),3)



        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        
        cv2.putText(img,f'FPS:{int(fps)}',(10,70),cv2.FONT_HERSHEY_COMPLEX,
              1,(0,0,255),3)

        cv2.imshow("Image", img)
        cv2.waitKey(1)
if __name__ == "__main__":
    main()
# HandTrackingModule####################################





# cap=cv2.VideoCapture(0)
# cap.set(3,1240)  #Can be altered according to ones need
# cap.set(4,760)    #Can be altered according to ones need
# pTime = 0
# detector = handDetector(detectionCon=0.7)

# while True:
#     success, img = cap.read()
#     img = detector.findHands(img)
#     lmlist = detector.findPosition(img,draw=True)
#     if len(lmList) != 0:
#         print(lmList[4])
#         x1,y1 = lmlist[4][1], lmlist[4][2]
#         x1,y1 = lmlist[8][1], lmlist[8][2]
#         cv2.circle(img,(x1,y1),15,(255,0,255),cv2.FILLED)
#         cv2.circle(img,(x2,y2),15,(255,0,255),cv2.FILLED)

        
    
#     cTime = time.time()
#     fps = 1/(cTime-pTime)
#     pTime = cTime
    
#     cv2.imshow("Img",img)
#     cv2.waitKey(1)

In [ ]:
(-96.0, 0.0, 0.125)

In [ ]:
220 and 20